<a href="https://colab.research.google.com/github/rickiepark/fine-tuning-llm/blob/main/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4장 데이터셋 포맷팅

### 스포일러

이 장에서는 다음과 같은 내용을 배웁니다.

- 적절한 채팅 템플릿의 중요성을 이해합니다.
- 사용자 정의 포맷팅 함수와 템플릿을 포함해 몇 가지 포맷팅 옵션에 대해 논의합니다.
- 토크나이저와 모델의 임베딩 층을 설정합니다.
- 패킹(packing)된 데이터셋과 데이터 로딩을 위한 다양한 데이터 콜레이터(data collator)를 살펴봅니다.

### 패키지 설치

In [ ]:
!pip install transformers==4.55.2 peft==0.17.0 accelerate==1.10.0 trl==0.21.0 datasets==4.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 라이브러리 임포트

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, \
    DataCollatorForLanguageModeling, DataCollatorWithPadding, \
    DataCollatorWithFlattening, BitsAndBytesConfig
from trl import setup_chat_format
from trl.data_utils import pack_dataset
from trl.extras.dataset_formatting import FORMAT_MAPPING, conversations_formatting_function
from compatibility_functions import DataCollatorForCompletionOnlyLM

### 목표

LLM에게 데이터셋의 구조와 단서를 제공하기 위해 포맷팅을 합니다. 적절한 태그와 특수 토큰으로 각 구성 요소(사용자 프롬프트와 모델이 완성할 텍스트)를 감싸서 모델의 동작을 쉽게 조정(예를 들면 지시 미세 튜닝)할 수 있습니다.

### 포맷팅의 핵심

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/base_prompt.png?raw=True)
<center>그림 4.1 베이스 모델의 다음 토큰 예측</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/fine_tuned_prompt.png?raw=True)
<center>그림 4.2 응답 템플릿을 사용하는 미세 튜닝된 모델</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/chat_prompt_new.png?raw=True)
<center>그림 4.3 채팅 템플릿을 사용하는 채팅 모델</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/chat_example_new.png?raw=True)
<center>그림 4.4 채팅 템플릿의 일반적인 구조</center>

### 이전 장에서

In [ ]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
compute_dtype = (torch.bfloat16 if supported else torch.float32)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=compute_dtype
)

model_q4 = AutoModelForCausalLM.from_pretrained("facebook/opt-350m",
                                                device_map='cuda:0',
                                                torch_dtype=compute_dtype,
                                                quantization_config=nf4_config)

model_q4 = prepare_model_for_kbit_training(model_q4)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model_q4, config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### 템플릿 적용하기

****
**"템플릿 적용하기" 절의 요약**

데이터셋 포맷팅에 세 가지 옵션이 있습니다:
1. 데이터셋의 포맷이 `STTrainer` 클래스가 지원하는 두 개의 포맷 중 하나인 경우
   - 토크나이저가 채팅 템플릿을 가지고 있어야 합니다.
   - 포맷팅 함수를 정의하거나 훈련 전에 데이터셋을 포맷팅할 필요가 없습니다.
2. 사용자 정의 포맷팅 함수를 사용하고 싶은 경우(BYOFF 절 참고)
   - `SFTTrainer` 클래스(5장 참조)의 `formatting_func` 매개변수로 사용자 정의 함수를 전달해야 합니다.
   - 사용자 정의 포맷팅 함수가 배치 데이터를 다룰 수 있어야 합니다.
     - `batched=True`로 데이터셋의 `map()` 메서드를 호출하여 테스트하세요.
    - 훈련 전에 데이터셋에 이 함수를 적용할 필요는 없습니다.
    - 토크나이저가 이미 채팅 템플릿을 가지고 있는 경우
      - 사용자 정의 함수에서 `apply_chat_template()` 메서드를 호출할 수 있습니다.
      - 템플릿의 일반적인 포맷(지시 템플릿과 응답 템플릿)을 고수하세요.
      - 템플릿에 `EOS` 토큰이 포함되어 있지 않다면 포맷팅된 출력 끝에 `EOS` 토큰을 추가할 수 있습니다.
   - 토크나이저가 채팅 템플릿을 가지고 있지 않은 경우
     - 지시 템플릿과 응답 템플릿을 포함하여 일반적인 포맷을 자유롭게 정의할 수 있습니다('고급 방법 - BYOT' 절 참고).
3. 데이터셋이 이미 포맷팅된 경우
   - `SFTTrainer` 클래스(5장 참조)의 `dataset_text_field` 매개변수에 포맷팅된 데이터를 담고 있는 열을 전달해야 합니다.
   - 사용자 정의 포맷팅 함수를 사용하여 데이터셋을 전처리 하더라도 훈련 클래스는 이를 사용하지 않습니다.
   - 데이터가 토크나이저의 템플릿과 호환되는지 확인하세요.
****

In [ ]:
tokenizer_phi = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct")
print(tokenizer_phi.chat_template)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %}


In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful AI assistant.'},
    {'role': 'user', 'content': 'What is the capital of Argentina?'},
    {'role': 'assistant', 'content': 'Buenos Aires.'}
]

formatted = tokenizer_phi.apply_chat_template(conversation=messages,
                                          tokenize=False,
                                          add_generation_prompt=False)
print(formatted)

<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
What is the capital of Argentina?<|end|>
<|assistant|>
Buenos Aires.<|end|>
<|endoftext|>


In [ ]:
inference_input = tokenizer_phi.apply_chat_template(conversation=messages[:-1],
                                          tokenize=False,
                                          add_generation_prompt=True)
print(inference_input)

<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
What is the capital of Argentina?<|end|>
<|assistant|>



#### 지원 포맷

In [ ]:
conversation_ds = Dataset.from_list([{'messages': messages}])
conversation_ds.features

{'messages': List({'content': Value('string'), 'role': Value('string')})}

In [ ]:
FORMAT_MAPPING['chatml'] == conversation_ds.features['messages']

True

In [ ]:
formatting_func = conversations_formatting_function(tokenizer_phi, messages_field='messages')

print(formatting_func(conversation_ds[0]))

<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
What is the capital of Argentina?<|end|>
<|assistant|>
Buenos Aires.<|end|>
<|endoftext|>


```python
# 대화 포맷을 위한 포맷팅 함수
def format_dataset(examples):
    if isinstance(examples[messages_field][0], list):
        output_texts = []
        for i in range(len(examples[messages_field])):
            output_texts.append(tokenizer.apply_chat_template(examples[messages_field][i], tokenize=False))
        return output_texts
    else:
        return tokenizer.apply_chat_template(examples[messages_field], tokenize=False)
```

**중요 업데이트**: 안타깝게도 최근 버전의 `trl` 라이브러리는 (`prompt`와 `completion` 열을 필요로 하는) 지시 포맷을 더이상 지원하지 않습니다. 이로 인해 채팅 템플릿이 올바르게 적용되지 않습니다. 이 문제를 피하기 위해 대화 포맷을 사용하는 것이 좋습니다.

만약 데이터셋이 지시 포맷으로 구성되어 있다면 다음에 나오는 `format_dataset()` 함수를 사용해 손쉽게 대화 포맷으로 바꿀 수 있습니다. 이 함수는 이전 버전의 `trl` 패키지를 참고하여 만들었습니다.

In [ ]:
def format_dataset(examples):
    if isinstance(examples["prompt"], list):
        output_texts = []
        for i in range(len(examples["prompt"])):
            converted_sample = [
                {"role": "user", "content": examples["prompt"][i]},
                {"role": "assistant", "content": examples["completion"][i]},
            ]
            output_texts.append(converted_sample)
        return output_texts
    else:
        converted_sample = [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
        return tokenizer.apply_chat_template(converted_sample, tokenize=False)

In [ ]:
batch_prompts_completions = {
    'prompt': ['What is the capital of Argentina?',
               'What is the capital of the United States?'],
    'completion': ['Buenos Aires.',
                    'Washington D.C.']
}

In [ ]:
batch_messages = format_dataset(batch_prompts_completions)['messages']
batch_messages

#### BYOFF (Bring Your Own Formatting Function)

In [ ]:
def byo_formatting_func1(examples):
    messages = examples["messages"]
    output_texts = tokenizer_phi.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return output_texts

In [ ]:
ds_msg = Dataset.from_dict({'messages': batch_messages})
ds_msg.map(lambda v: tokenizer_phi(byo_formatting_func1(v)), batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['messages', 'input_ids', 'attention_mask'],
    num_rows: 2
})

In [ ]:
def byo_formatting_func2(examples):
    response_template = '### Answer:'
    text = f"### Question: {examples['prompt']}\n{response_template} {examples['completion']}"
    text += tokenizer_phi.eos_token
    return text

In [ ]:
ds_prompt = Dataset.from_dict(batch_prompts_completions)
print(byo_formatting_func2(ds_prompt[0]))

### Question: What is the capital of Argentina?
### Answer: Buenos Aires.<|endoftext|>


In [ ]:
# 이 코드는 예외를 일으킵니다.
ds_prompt.map(lambda v: tokenizer_phi(byo_formatting_func2(v)), batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

ArrowInvalid: Column 2 named input_ids expected length 2 but got length 44

In [ ]:
def byo_formatting_func3(examples):
    output_texts = []
    response_template = '### Answer:'
    for i in range(len(examples['prompt'])):
        text = f"### Question: {examples['prompt'][i]}\n {response_template} {examples['completion'][i]}"
        text += tokenizer_phi.eos_token
        output_texts.append(text)
    return output_texts

In [ ]:
ds_prompt.map(lambda v: tokenizer_phi(byo_formatting_func3(v)), batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 2
})

#### BYOFD (Bring Your Own Formatted Data)

In [ ]:
def byofd_formatting_func(examples):
    messages = examples["messages"]
    output_texts = tokenizer_phi.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return {'text': output_texts}

In [ ]:
formatted_ds = ds_msg.map(byofd_formatting_func, batched=True)
formatted_ds['text']

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Column(['<|user|>\nWhat is the capital of Argentina?<|end|>\n<|assistant|>\nBuenos Aires.<|end|>\n<|endoftext|>', '<|user|>\nWhat is the capital of the United States?<|end|>\n<|assistant|>\nWashington D.C.<|end|>\n<|endoftext|>'])

#### 결론

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/formatting_flow.png?raw=True)

<center>그림 4.5 - 적절한 포맷팅 방법 선택하기</center>

### 토크나이저

****
**"토크나이저" 절의 요약**
- 토크나이저의 어휘사전은 일반적으로 모델의 임베딩 층 크기보다 작습니다.
  - 크기 차이는 임베딩 층의 크기를 바꾸지 않고도 새로운 토큰을 추가할 수 있는 빈 슬롯 때문입니다.
  - 효율적인 메모리 할당을 위해 임베딩 층의 크기는 2의 거듭제곱의 배수(32, 64 등)인 경우가 많습니다.
- `EOS` 토큰은 다른 것 말고 텍스트의 끝을 나타내는데만 사용해야 합니다.
  - 패딩을 위해 `EOS` 토큰을 사용하면 토큰 생성이 끊임없이 계속되는 문제가 발생할 수 있습니다.
- `PAD` 토큰을 정의하지 않는 경우가 많지만 여전히 이 토큰이 필요합니다.
  - `EOS` 토큰을 `PAD` 토큰으로 할당하지 마세요.
  - `UNK` 토큰이 정의되어 있다면 이를 `PAD` 토큰으로 할당해도 괜찮습니다.
  - `UNK` 토큰이 정의되어 있지 않다면 `PAD` 토큰을 위해 새로운 특수 토큰을 만드세요.
  - 주의: `PAD` 토큰을 정의하지 않은 채로 두면 많은 라이브러리에서 기본적으로 `EOS` 토큰을 패딩 토큰으로 할당합니다!!
- 생성 모델의 경우 패딩은 왼쪽에 추가되어야 합니다.
  - 오른쪽에 패딩하면 모델이 패딩 토큰의 시퀀스를 생성하도록 훈련됩니다.
  - `SFTTrainer` 클래스에서 보고된 오버플로 문제 때문에 많은 튜토리얼에서 `tokenizer.padding_side='right'`를 사용합니다.
    - 표준 패딩이 아니라 패킹이나 패킹 역할을 하는 콜레이터("패킹된 데이터셋" 절 참조)를 사용하는 경우에만 괜찮습니다.
- 새로운 특수 토큰을 만든다면 (임베딩 층의 빈 슬롯을 사용하기 때문에) 이론적으로 임베딩 층도 미세 튜닝해야 합니다.
  - 실제로는 임베딩을 동결하더라도 모델이 잘 동작할 수 있습니다.
  - (해당 임베딩을 훈련하지 않았으므로) 새로운 토큰 표현이 랜덤하지만, 모델의 훈련 가능한 다른 부분이 이런 임베딩을 있는 그대로 사용하는 방법을 학습할 수 있습니다.
****

In [ ]:
tokenizer_phi = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct")
config_phi = AutoConfig.from_pretrained("microsoft/phi-3-mini-4k-instruct", trust_remote_code=True)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
tokenizer_phi("Let's tokenize this sentence!")

{'input_ids': [2803, 29915, 29879, 5993, 675, 445, 10541, 29991], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

#### 어휘사전

In [ ]:
len(tokenizer_phi), config_phi.vocab_size

(32011, 32064)

In [ ]:
sorted(tokenizer_phi.vocab.items(), key=lambda t: -t[1])[:11]

[('<|user|>', 32010),
 ('<|placeholder6|>', 32009),
 ('<|placeholder5|>', 32008),
 ('<|end|>', 32007),
 ('<|system|>', 32006),
 ('<|placeholder4|>', 32005),
 ('<|placeholder3|>', 32004),
 ('<|placeholder2|>', 32003),
 ('<|placeholder1|>', 32002),
 ('<|assistant|>', 32001),
 ('<|endoftext|>', 32000)]

In [ ]:
tokenizer_phi.eos_token, tokenizer_phi.eos_token_id

('<|endoftext|>', 32000)

#### 특수 토큰

In [ ]:
tokenizer_phi.all_special_tokens

['<s>', '<|endoftext|>', '<unk>']

In [ ]:
tokenizer_phi.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'pad_token': '<|endoftext|>'}

In [ ]:
tokenizer_phi.cls_token, tokenizer_phi.sep_token, tokenizer_phi.mask_token

(None, None, None)

In [ ]:
tokenizer_phi.add_special_tokens({'cls_token': '<cls>', 'sep_token': '<sep>', 'mask_token': '<mask>'})
tokenizer_phi.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'sep_token': '<sep>',
 'pad_token': '<|endoftext|>',
 'cls_token': '<cls>',
 'mask_token': '<mask>'}

In [ ]:
sorted(tokenizer_phi.vocab.items(), key=lambda t: -t[1])[:13]

[('<mask>', 32013),
 ('<sep>', 32012),
 ('<cls>', 32011),
 ('<|user|>', 32010),
 ('<|placeholder6|>', 32009),
 ('<|placeholder5|>', 32008),
 ('<|end|>', 32007),
 ('<|system|>', 32006),
 ('<|placeholder4|>', 32005),
 ('<|placeholder3|>', 32004),
 ('<|placeholder2|>', 32003),
 ('<|placeholder1|>', 32002),
 ('<|assistant|>', 32001)]

#### `EOS` 토큰

In [ ]:
tokenizer_phi.pad_token = tokenizer_phi.unk_token
tokenizer_phi.pad_token_id = tokenizer_phi.unk_token_id

tokenizer_phi.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'sep_token': '<sep>',
 'pad_token': '<unk>',
 'cls_token': '<cls>',
 'mask_token': '<mask>'}

```python
# 수정된 패딩 토큰을 위해 모델 설정을 업데이트합니다.
if getattr(model, "config", None) is not None:
    model.config.pad_token_id = tokenizer_phi.pad_token_id
if (getattr(model, "generation_config", None) s not None):
    model.config.pad_token_id = tokenizer_phi.pad_token_id
```

#### `PAD` 토큰

In [ ]:
tokenizer_phi.pad_token, tokenizer_phi.padding_side

('<unk>', 'left')

### 데이터 콜레이터

****
**"데이터 콜레이터" 절의 요약**
- `SFTTrainer` 클래스(5장 참조)의 `data_collator` 매개변수를 지정할 수 있습니다.
- `DataCollatorForLanguageModeling`가 `SFTTrainer` 클래스의 기본 콜레이터입니다.
  - 자동으로 토큰 ID를 레이블로 복제합니다.
  - 모델이 자동으로 처리하므로 레이블을 이동시키지 않습니다.
  - 전체 텍스트(프롬프트와 완성)를 레이블에 포함시키므로 지시 미세 튜닝에 이상적입니다.
- 지시 모델이나 채팅 모델을 추가적으로 미세 튜닝한다면 `DataCollatorForCompletionOnlyLM`을 사용해 모델의 응답(완성)으로만 모델을 훈련할 수 있습니다.
  - 이 콜레이터도 토큰 ID를 레이블로 복제하지만 프롬프트 토큰에 해당하는 ID는 -100으로 마스킹합니다.
  - 단일 상호작용(한 쌍의 프롬프트와 완성)에서는 응답 템플릿만으로 완성의 위치를 찾을 수 있습니다.
  - 다중 상호작용(여러 쌍의 프롬프트와 완성)에서는 프롬프트 토큰을 찾고 마스킹하기 위해 지시 템플릿과 응답 템플릿이 모두 필요합니다.
****

In [ ]:
dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
# 프롬프트/완성 쌍을 대화 메시지로 변환합니다.
dataset = dataset.map(format_dataset)
dataset = dataset.remove_columns(["prompt", "completion", "translation"])
len(dataset), dataset[0]

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentences.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/720 [00:00<?, ? examples/s]

(720,
 {'prompt': 'The birch canoe slid on the smooth planks.',
  'completion': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.'})

In [ ]:
formatting_func = conversations_formatting_function(tokenizer_phi,
                                                    messages_field='messages')
dataset = dataset.map(lambda row: {'text': formatting_func(row)},
                      batched=True, batch_size=32)
sequences = dataset['text']
print(sequences[:2])

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

['<|user|>\nThe birch canoe slid on the smooth planks.<|end|>\n<|assistant|>\nOn the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>\n<|endoftext|>', '<|user|>\nGlue the sheet to the dark blue background.<|end|>\n<|assistant|>\nGlue the sheet to the dark blue background, you must.<|end|>\n<|endoftext|>']


In [ ]:
tokenized_dataset = dataset.map(lambda row: tokenizer_phi(row['text']))
tokenized_dataset = tokenized_dataset.select_columns(['input_ids'])

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

#### `DataCollatorWithPadding`

In [ ]:
pad_collator = DataCollatorWithPadding(tokenizer_phi)
pad_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=pad_collator)
pad_batch = next(iter(pad_dloader))
pad_batch

{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         32010,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29889, 32007, 32001,  8467,   434,   278,  9869,   304,   278,  6501,
          7254,  3239, 29892,   366,  1818, 29889, 32007, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

#### 레이블은 어디 있나요?

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/shift_labels.png?raw=True)

<center>그림 4.6 입력과 한 위치 이동한 레이블</center>

#### `DataCollatorForLanguageModeling`

In [ ]:
lm_collator = DataCollatorForLanguageModeling(tokenizer_phi, mlm=False)
lm_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=lm_collator)
lm_batch = next(iter(lm_dloader))
lm_batch

{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         32010,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29889, 32007, 32001,  8467,   434,   278,  9869,   304,   278,  6501,
          7254,  3239, 29892,   366,  1818, 29889, 32007, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
   

#### `DataCollatorForCompletionOnlyLM`

In [ ]:
response_template = '<|assistant|>' # token id 32001
completion_collator = DataCollatorForCompletionOnlyLM(response_template=response_template,
                                                      tokenizer=tokenizer_phi)
completion_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=completion_collator)
completion_batch = next(iter(completion_dloader))
completion_batch

{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         32010,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29889, 32007, 32001,  8467,   434,   278,  9869,   304,   278,  6501,
          7254,  3239, 29892,   366,  1818, 29889, 32007, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
   

In [ ]:
labels = completion_batch['labels'][0]
valid_tokens = (labels >= 0)
tokenizer_phi.decode(labels[valid_tokens])

'On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|><|endoftext|>'

##### 다중 상호작용

In [ ]:
dummy_chat = """<|user|>Hello
<|assistant|>How are you?
<|user|>I'm fine! You?
<|assistant|>I'm fine too!
<|endoftext|>"""

dummy_ds = Dataset.from_dict({'text': [dummy_chat]})
dummy_ds = dummy_ds.map(lambda row: tokenizer_phi(row['text'])).select_columns(['input_ids'])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
completion_dloader = DataLoader(dummy_ds, batch_size=1, collate_fn=completion_collator)
completion_batch = next(iter(completion_dloader))
completion_batch

{'input_ids': tensor([[32010, 15043,    13, 32001,  1128,   526,   366, 29973,    13, 32010,
           306, 29915, 29885,  2691, 29991,   887, 29973,    13, 32001,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]])}

In [ ]:
labels = completion_batch['labels']
tokenizer_phi.decode(labels[labels >= 0])

"I'm fine too!\n<|endoftext|>"

In [ ]:
instruction_template = '<|user|>'
response_template = '<|assistant|>'
completion_collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,
                                                      response_template=response_template,
                                                      tokenizer=tokenizer_phi)
completion_dloader = DataLoader(dummy_ds, batch_size=1, collate_fn=completion_collator)
completion_batch = next(iter(completion_dloader))
completion_batch

{'input_ids': tensor([[32010, 15043,    13, 32001,  1128,   526,   366, 29973,    13, 32010,
           306, 29915, 29885,  2691, 29991,   887, 29973,    13, 32001,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  1128,   526,   366, 29973,    13,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]])}

In [ ]:
labels = completion_batch['labels']
tokenizer_phi.decode(labels[labels >= 0])

"How are you?\n I'm fine too!\n<|endoftext|>"

#### 레이블 이동

```python
if labels is not None:
    # 모델 병렬화를 위해 레이블을 올바른 장치로 이동시킵니다.
    labels = labels.to(lm_logits.device)
    # 다음 토큰 예측을 수행하므로 예측 점수와 입력 아이디를 하나씩 이동시킵니다.
    shift_logits = lm_logits[:, :-1, :].contiguous()
    labels = labels[:, 1:].contiguous()
    loss_fct = CrossEntropyLoss()
    lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))
```

### 패킹된 데이터셋

****
**"패킹된 데이터셋" 절의 요약**
- 패킹은 시퀀스를 연결하여 동일한 크기의 청크로 나눕니다.
  - 패딩 토큰이 사용되지 않습니다.
  - 각 청크의 길이는 모델의 최대 시퀀스 길이를 넘어서는 안됩니다.
- 패킹은 기본적으로 `SFTTrainer`에서 지원합니다.
  - `packing` 매개변수를 `True`로 지정합니다.
  - `pack_dataset()` 함수를 사용하여 패킹을 처리합니다.
  - `packing_strategy`를 `wrapped`로 지정하여 원본 패킹 동작을 근사할 수 있습니다.
  - 기본적으로 패킹과 콜레이터를 동시에 사용할 수 없습니다.
- 일부 콜레이터는 효과적으로 시퀀스를 패킹할 수 있습니다.
  - 이 경우 `packing` 매개변수를 `False`로 지정해야 하며 콜레이터가 패킹을 수행합니다.
  - `DataCollatorWithFlattening`는 `DataCollatorForLanguageModeling`의 패킹 버전입니다.
  - `DataCollatorForCompletionOnlyLM`에는 완성 전용 콜레이터가 패킹 같은 기능을 수행하도록 만드는 새로운 매개변수(`padding_free`)가 있습니다.
  - 특정 모델(예를 들면, Llama, Phi, Mistral, Gemma, OLMo 등)은 플래시 어텐션 2로 이런 콜레이터를 지원합니다.
    - 이런 모델은 `position_ids`를 사용하여 패킹된 원본 시퀀스 사이의 경계를 표시합니다.
****

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/packed_seq.png?raw=True)

<center>그림 4.7 패킹된 시퀀스</center>

In [ ]:
sequences = dataset['text']
print(sequences[:2])

['<|user|>\nThe birch canoe slid on the smooth planks.<|end|>\n<|assistant|>\nOn the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>\n<|endoftext|>', '<|user|>\nGlue the sheet to the dark blue background.<|end|>\n<|assistant|>\nGlue the sheet to the dark blue background, you must.<|end|>\n<|endoftext|>']


In [ ]:
packed_dataset = pack_dataset(tokenized_dataset, seq_length=64,
                              strategy='wrapped')
packed_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 351
})

In [ ]:
input_ids = packed_dataset['input_ids']
tokenizer_phi.decode(input_ids[0])

'<|user|> The birch canoe slid on the smooth planks.<|end|><|assistant|> On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|><|endoftext|><|endoftext|><|user|> Glue the sheet to the dark blue background.<|end|><|assistant|> Glue the sheet to the dark blue background, you must'

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/packing_flow.png?raw=True)

<center>그림 4.8 올바른 데이터 설정 선택하기</center>

#### 패킹을 위한 콜레이터

##### `DataCollatorWithFlattening`

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/collator_flat.png?raw=True)

<center>그림 4.9 패킹 유사 콜레이터</center>

In [ ]:
flat_collator = DataCollatorWithFlattening()
flat_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=flat_collator)
flat_batch = next(iter(flat_dloader))
flat_batch

{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
          10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
           1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
           3869, 29892,   298, 21478,  1758, 29889, 32007, 32000, 32010,  8467,
            434,   278,  9869,   304,   278,  6501,  7254,  3239, 29889, 32007,
          32001,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
          29892,   366,  1818, 29889, 32007, 32000]]),
 'labels': tensor([[ -100,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
          10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
           1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
           3869, 29892,   298, 21478,  1758, 29889, 32007, 32000,  -100,  8467,
            434,   278,  9869,   304,   278,  6501,  7254,  3239, 29889, 32007,
          32001,  8467,   434,   278,  986

In [ ]:
flat_batch['input_ids'].shape, flat_batch['position_ids'].max() + 1

(torch.Size([1, 66]), tensor(38))

##### `DataCollatorForCompletionOnlyLM`

In [ ]:
response_template = '<|assistant|>'
completion_nopad_collator = DataCollatorForCompletionOnlyLM(response_template=response_template,
                                                            tokenizer=tokenizer_phi,
                                                            padding_free=True)
completion_nopad_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=completion_nopad_collator)
completion_nopad_batch = next(iter(completion_nopad_dloader))
completion_nopad_batch

{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000, 32010,  8467,
           434,   278,  9869,   304,   278,  6501,  7254,  3239, 29889, 32007,
         32001,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29892,   366,  1818, 29889, 32007, 32000]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  8467,   434,   278,  9869,   304,   

### 고급 방법: BYOT (Bring Your Own Template)

****
**"고급 방법: BYOT" 절의 요약**

- 모든 템플릿은 응답 템플릿을 정의해야 하며, 이상적으로는 EOS 토큰으로 끝나야 합니다.
- 토크나이저의 `EOS`, `PAD`, `UNK` 토큰을 다시 확인하세요.
  - `EOS` 토큰은 `PAD` 및 `UNK` 토큰과 달라야 합니다.
  - `PAD` 및 `UNK` 토큰은 같을 수 있습니다.
- 꼭 필요할 때 임베딩 층의 크기를 바꿉니다("빈 슬롯"이 모두 사용된 경우).
  - 모델의 `resize_token_embeddings()`를 호출할 때, `pad_to_multiple_of` 매개변수를 사용해 크기가 2의 거듭제곱의 배수로 유지되도록 합니다.
- 진자 템플릿을 직접 만들고 싶지 않다면 ChatML과 같은 기본 템플릿을 사용할 수 있습니다. `trl` 패키지의 `setup_chat_format()` 함수를 사용할 수 있지만 몇 가지 단점이 있습니다.
  - `EOS` 토큰을 `PAD` 토큰에 할당합니다(나중에 수동으로 수정해야 합니다).
  - 더 짧게 만들기 위해서라도 모델의 임베딩 층 크기를 기본적으로 조정합니다(적절한 `resize_to_multiple_of`를 선택하면 크기 변경을 피할 수 있습니다).
- 토크나이저를 위해 진자 템플릿을 만드는 대신 포맷팅 함수를 사용하여 사용자 정의 템플릿을 정의하고 적용할 수 있습니다.
  - `SFTTrainer` 클래스(5장 참조)에서 `formatting_func`을 지정하면 토크나이저가 채팅 템플릿을 가지고 있을 필요가 없습니다.
- 응답 템플릿을 신중하게 선택하세요.
  - 일반 단어(예: "## Answer:")를 사용하면 문제가 발생할 수 있습니다. 일부 토크나이저는 문맥에 의존적이어서 응답 템플릿을 여러 토큰으로 분할할 수 있기 때문입니다.
  - 응답 템플릿을 위해 추가적인 특수 토큰을 만드는 것이 더 안전합니다. 이렇게 하면 응답 템플릿이 단일 토큰으로 인코딩되기 때문입니다.
****

#### 채팅 템플릿

In [ ]:
model_opt = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer_opt = AutoTokenizer.from_pretrained("facebook/opt-350m")

print(tokenizer_opt.chat_template)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

None


In [ ]:
tokenizer_opt.special_tokens_map

{'bos_token': '</s>',
 'eos_token': '</s>',
 'unk_token': '</s>',
 'pad_token': '<pad>'}

**ChatML**
****

[ChatML](https://github.com/openai/openai-python/blob/release-v0.28.0/chatml.md)은 채팅 마크업 언어(Chat Markup Language)의 약어로 오픈AI에서 개발했습니다:

_____
"_전통적으로 GPT 모델은 구조화되지 않은 텍스트를 사용했습니다. ChatGPT 모델은 대신에 채팅 마크업 언어(줄여서 ChatML)이라는 구조적인 포맷을 기대합니다. ChatML 문서는 일련의 메시지로 구성됩니다._"
_____

앞서 소개한 대화 포맷과 비슷하게 각 메시지는 참여자의 역할과 그에 해당하는 콘텐츠로 구성됩니다. ChatML의 진자 템플릿은 다음과 같습니다:

```
{% for message in messages %}
  {{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}
{% endfor %}
```
****

In [ ]:
len(tokenizer_opt)

50265

In [ ]:
model_opt.config.vocab_size

50272

In [ ]:
def get_multiple_of(vocab_size):
    return 2**(bin(vocab_size)[::-1].find('1'))

pad_to_multiple_of = get_multiple_of(model_opt.config.vocab_size)
pad_to_multiple_of

32

In [ ]:
model_opt.resize_token_embeddings(len(tokenizer_opt),
                                  pad_to_multiple_of=pad_to_multiple_of)

Embedding(50272, 512, padding_idx=1)

In [ ]:
def modify_tokenizer(tokenizer,
                     alternative_bos_token='<|im_start|>',
                     alternative_unk_token='<unk>',
                     special_tokens=None,
                     tokens=None):
    eos_token, bos_token = tokenizer.eos_token, tokenizer.bos_token
    pad_token, unk_token = tokenizer.pad_token, tokenizer.unk_token

    # BOS 토큰은 EOS 토큰과 달라야 합니다.
    if bos_token == eos_token:
        bos_token = alternative_bos_token

    # UNK 토큰은 EOS 토큰과 달라야 합니다.
    if unk_token == eos_token:
        unk_token = alternative_unk_token

    # PAD 토큰은 EOS 토큰과 달라야 합니다.
    # 하지만 UNK 토큰과는 같을 수 있습니다.
    if pad_token == eos_token:
        pad_token = unk_token

    assert bos_token != eos_token, "다른 BOS 토큰을 선택하세요."
    assert unk_token != eos_token, "다른 UNK 토큰을 선택하세요."

    # BOS, PAD, UNK 토큰을 위한 딕셔너리를 만듭니다.
    # EOS 토큰은 원래 정의된 대로 유지합니다.
    special_tokens_dict = {'bos_token': bos_token,
                           'pad_token': pad_token,
                           'unk_token': unk_token}

    # 새로운 특수 토큰을 추가합니다.
    if special_tokens is not None:
        if isinstance(special_tokens, list):
            special_tokens_dict.update({'additional_special_tokens': special_tokens})

    tokenizer.add_special_tokens(special_tokens_dict)

    # 새로운 일반 토큰을 추가합니다.
    if tokens is not None:
        if isinstance(tokens, list):
            tokenizer.add_tokens(tokens)

    return tokenizer

In [ ]:
def jinja_template(tokenizer):
    return ("{% for message in messages %}"
            f"{{{{'{tokenizer.bos_token}' + message['role'] + '\n' + message['content'] + '{tokenizer.eos_token}' + '\n'}}}}"
            "{% endfor %}"
            "{% if add_generation_prompt %}"
            f"{{{{ '{tokenizer.bos_token}assistant\n' }}}}"
            "{% endif %}")

def add_template(tokenizer, chat_template=None):
    # 채팅 템플릿이 주어지지 않으면 BOS와 EOS 토큰을 사용해
    # 채팅 템플릿을 만듭니다.
    if chat_template is None:
        chat_template = jinja_template(tokenizer)

    # 채팅 템플릿을 토크나이저에 할당합니다.
    tokenizer.chat_template = chat_template

    return tokenizer

In [ ]:
def get_multiple_of(vocab_size):
    return 2**(bin(vocab_size)[::-1].find('1'))

def modify_model(model, tokenizer):
    # 새로운 토크나이저의 크기가 어휘사전 크기를 초과한다면
    # 같은 배수가 되도록 유지하면서 크기를 바꿉니다.
    if len(tokenizer) > model.config.vocab_size:
        pad_to_multiple_of = get_multiple_of(model.vocab_size)
        model.resize_token_embeddings(len(tokenizer),
                                      pad_to_multiple_of=pad_to_multiple_of)

    # 모델 설정의 토큰 ID를 업데이트합니다.
    if getattr(model, "config", None) is not None:
        model.config.pad_token_id = tokenizer.pad_token_id
        model.config.bos_token_id = tokenizer.bos_token_id
        model.config.eos_token_id = tokenizer.eos_token_id
    if getattr(model, "generation_config", None) is not None:
        model.generation_config.bos_token_id = tokenizer.bos_token_id
        model.generation_config.eos_token_id = tokenizer.eos_token_id
        model.generation_config.pad_token_id = tokenizer.pad_token_id

    return model

In [ ]:
tokenizer_opt = modify_tokenizer(tokenizer_opt)
tokenizer_opt = add_template(tokenizer_opt)
model_opt = modify_model(model_opt, tokenizer_opt)

In [ ]:
tokenizer_opt.special_tokens_map

{'bos_token': '<|im_start|>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>'}

In [ ]:
len(tokenizer_opt)

50266

In [ ]:
tokenizer_opt.convert_ids_to_tokens(50265)

'<|im_start|>'

In [ ]:
model_opt.get_input_embeddings()

Embedding(50272, 512, padding_idx=1)

In [ ]:
print(tokenizer_opt.chat_template)

{% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '</s>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


In [ ]:
messages = ds_msg['messages'][0]
print(tokenizer_opt.apply_chat_template(messages, tokenize=False))

<|im_start|>user
What is the capital of Argentina?</s>
<|im_start|>assistant
Buenos Aires.</s>



#### 사용자 정의 템플릿

In [ ]:
model_opt = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer_opt = AutoTokenizer.from_pretrained("facebook/opt-350m")

response_template = '##[YODA]##>'
tokenizer_opt = modify_tokenizer(tokenizer_opt, special_tokens=[response_template])
model_opt = modify_model(model_opt, tokenizer_opt)

In [ ]:
def formatting_func_builder(response_template):
    def formatting_func(examples, add_generation_prompt=False):
        output_texts = []
        for i in range(len(examples['prompt'])):
            text = f"{examples['prompt'][i]}"
            try:
                text += f" {response_template} {examples['completion'][i]}{tokenizer_opt.eos_token}"
            except KeyError:
                if add_generation_prompt:
                    text += f" {response_template} "
            output_texts.append(text)
        return output_texts
    return formatting_func

yoda_formatting_func = formatting_func_builder(response_template)
yoda_formatting_func

<function __main__.formatting_func_builder.<locals>.formatting_func(examples, add_generation_prompt=False)>

In [ ]:
dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")

formatted_seqs = yoda_formatting_func(dataset)
formatted_seqs[0]

'The birch canoe slid on the smooth planks. ##[YODA]##> On the smooth planks, the birch canoe slid. Yes, hrrrm.</s>'

In [ ]:
tokenizer_opt(formatted_seqs[0])

{'input_ids': [2, 133, 23629, 611, 31728, 13763, 15, 5, 6921, 563, 2258, 4, 1437, 50266, 374, 5, 6921, 563, 2258, 6, 5, 23629, 611, 31728, 13763, 4, 3216, 6, 1368, 28015, 22900, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer_opt.convert_ids_to_tokens(50266)

'##[YODA]##>'

In [ ]:
yoda_formatting_func({'prompt': ['The Force is strong in you.',
                                 'I am your father!']},
                     add_generation_prompt=True)

['The Force is strong in you. ##[YODA]##> ', 'I am your father! ##[YODA]##> ']

#### 특수 토큰 만세!

In [ ]:
from huggingface_hub import login
login()

In [ ]:
tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer_llama.pad_token = tokenizer_llama.unk_token
tokenizer_llama.pad_token_id = tokenizer_llama.unk_token_id

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
prompt = """### User: Hello\n\n### Assistant: Hi, how can I help you?"""
print(prompt)

### User: Hello

### Assistant: Hi, how can I help you?


In [ ]:
tokens = tokenizer_llama.tokenize(prompt, add_special_tokens=False)
token_ids = tokenizer_llama.encode(prompt, add_special_tokens=False)
list(zip(tokens, token_ids))[6:11]

[('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

In [ ]:
response_template = "### Assistant:"
tokens = tokenizer_llama.tokenize(response_template, add_special_tokens=False)
token_ids = tokenizer_llama.encode(response_template, add_special_tokens=False)
list(zip(tokens, token_ids))

[('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

In [ ]:
dummy_ds = Dataset.from_dict({'text': [prompt]})
dummy_tokenized = dummy_ds.map(lambda row: tokenizer_llama(row['text'])).select_columns(['input_ids'])

response_template = "### Assistant:"

bad_collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer_llama)
bad_dloader = DataLoader(dummy_tokenized, batch_size=1, collate_fn=bad_collator)
bad_batch = next(iter(bad_dloader))
bad_batch

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:153: UserWarning: Could not find response key `### Assistant:` in the following instance: <s> ### User: Hello

### Assistant: Hi, how can I help you?. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_length`.
  warnings.warn(


{'input_ids': tensor([[    1,   835,  4911, 29901, 15043,    13,    13,  2277, 29937,  4007,
         22137, 29901,  6324, 29892,   920,   508,   306,  1371,   366, 29973]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]])}

In [ ]:
modified_response_template = "\n### Assistant:"
tokens = tokenizer_llama.tokenize(modified_response_template, add_special_tokens=False)
token_ids = tokenizer_llama.encode(modified_response_template, add_special_tokens=False)
list(zip(tokens, token_ids))

[('▁', 29871),
 ('<0x0A>', 13),
 ('##', 2277),
 ('#', 29937),
 ('▁Ass', 4007),
 ('istant', 22137),
 (':', 29901)]

In [ ]:
fixed_token_ids = token_ids[2:]
fixed_collator = DataCollatorForCompletionOnlyLM(fixed_token_ids, tokenizer=tokenizer_llama)
fixed_dloader = DataLoader(dummy_tokenized, batch_size=1, collate_fn=fixed_collator)
fixed_batch = next(iter(fixed_dloader))
fixed_batch

{'input_ids': tensor([[    1,   835,  4911, 29901, 15043,    13,    13,  2277, 29937,  4007,
         22137, 29901,  6324, 29892,   920,   508,   306,  1371,   366, 29973]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  6324, 29892,   920,   508,   306,  1371,   366, 29973]])}

In [ ]:
response_template = "### Assistant:"
tokenizer_llama.add_special_tokens({'additional_special_tokens': [response_template]})

1

In [ ]:
dummy_tokenized = dummy_ds.map(lambda row: tokenizer_llama(row['text'])).select_columns(['input_ids'])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
special_collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer_llama)
special_dloader = DataLoader(dummy_tokenized, batch_size=1, collate_fn=special_collator)
special_batch = next(iter(special_dloader))
special_batch

{'input_ids': tensor([[    1,   835,  4911, 29901, 15043,    13,    13, 32000, 29871,  6324,
         29892,   920,   508,   306,  1371,   366, 29973]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 29871,  6324,
         29892,   920,   508,   306,  1371,   366, 29973]])}

### 다음 장에서는

채팅 템플릿은 야생의 LLM을 길들이고 인간과 적절한 대화를 할 수 있는 방법을 가르치기 위한 핵심입니다. 대화 사이에 큐 사인(또는 특수 토큰)을 적절하게 배치하면 명령 키워드에 따라 어떻게 응답해야 하는지 학습할 수 있습니다. 하지만 훈련 절차에 위험이 없는 것은 아닙니다. 활성화, 그레이디언트, 옵티마이저 모두 자신의 작업을 수행하기 위해 상당한 양의 RAM을 필요로 합니다. 메모리를 많이 소비하는 이런 구성 요소들을 만족시키려면 기술과 노력이 모두 필요합니다. 훈련 루프를 구성하려면 두려움을 떨쳐내야 합니다. 도전이 가득한 다음 장을 놓치지 마세요!